In [17]:
import ruamel.yaml
import os
import re

In [ ]:
dbt_base_path = ''
if not dbt_base_path:
    dbt_base_path = get_destination()

In [18]:
dbt_base_path = "/Users/tim/Desktop/arc-mercycorps-dbt"

In [19]:
yaml = ruamel.yaml.YAML()
yaml.indent(mapping=4, sequence=4, offset=2)
yaml.preserve_quotes = True

In [20]:
with open('../../arc-standard-dashboard-dbt-fake-data-generator/.github/workflows/dbt_run_on_merge.yml','r') as f:
    content = f.read()
    workflow_yml = yaml.load(content)

In [21]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(workflow_yml)

{   'jobs': {   'dbt_run': {   'runs-on': 'ubuntu-latest',
                               'steps': [ordereddict([('name', 'Run dbt cloud job'), ('run', 'RESPONSE=$(curl -v -H "Authorization:Token ${{ secrets.DBTCLOUDAPIKEY }}" -H "Content-Type:application/json" -d \'{"cause":"Triggered by Github Actions","git_branch":"main"}\' https://cloud.getdbt.com/api/v2/accounts/1713/jobs/121379/run/)\nSTATUSCODE=$(echo "$RESPONSE" | jq \'.status.code\')\necho "$RESPONSE" | jq \'.status.user_message\'\nif [[ $STATUSCODE != 200 ]]; then echo "$RESPONSE" && bash -c "exit 1"; fi\n')])]}},
    'name': 'Trigger dbt cloud job to prep views and build docs',
    'on': ordereddict([('push', ordereddict([('branches', ['main'])]))])}


In [22]:
starting_prompt = """You'll need to do some setup to have this util run.

First, you'll need to have a working dbt Github Repo.

Second, you'll need to have a dbtCloud project set up.
If you've never done this before, it's worth talking to an engineer.
Here are the docs if you'd like to try on your own:
https://docs.getdbt.com/docs/get-started/getting-started/set-up-dbt-cloud

Third, you need to set up an environment in dbt Cloud. Here are docs:
https://docs.getdbt.com/docs/get-started/getting-started/building-your-first-project/schedule-a-job#create-a-deployment-environment

Fourth, you need to create a job in dbt Cloud. Here are docs:
https://docs.getdbt.com/docs/get-started/getting-started/building-your-first-project/schedule-a-job#create-and-run-a-job

Finally, you'll need your dbt Cloud API key.
You can find it at the bottom of this page:
https://cloud.getdbt.com/next/settings/profile
"""

In [23]:
check_api_added_to_secrets = input("""Please add a secret to your dbt repo with the name DBTCLOUDAPIKEY
Instructions on how to do this below:
https://docs.github.com/en/codespaces/managing-codespaces-for-your-organization/managing-encrypted-secrets-for-your-repository-and-organization-for-github-codespaces
""")

Please add a secret to your dbt repo with the name DBTCLOUDAPIKEY
Instructions on how to do this below:
https://docs.github.com/en/codespaces/managing-codespaces-for-your-organization/managing-encrypted-secrets-for-your-repository-and-organization-for-github-codespaces
y


In [24]:
dbt_cloud_job_url = input("""Now please go to your cloud job in dbt Cloud, copy the URL, and paste it here.
It should look something like this: https://cloud.getdbt.com/next/deploy/1713/projects/150080/jobs/121379
""")

Now please go to your cloud job in dbt Cloud, copy the URL, and paste it here.
It should look something like this: https://cloud.getdbt.com/next/deploy/1713/projects/150080/jobs/121379
https://cloud.getdbt.com/next/deploy/1713/projects/105741/jobs/92226


In [25]:
dbt_cloud_api_run_url = re.sub("projects/d+/",
                               "",
                               dbt_cloud_job_url.replace("next/deploy/","api/v2/accounts/")
                              )+"/run/"

In [15]:
workflow_yml['jobs']['dbt_run']['steps'][0]['run'].replace('{dbt_cloud_api_run_url}',dbt_cloud_api_run_url)

'RESPONSE=$(curl -v -H "Authorization:Token ${{ secrets.DBTCLOUDAPIKEY }}" -H "Content-Type:application/json" -d \'{"cause":"Triggered by Github Actions","git_branch":"main"}\' https://cloud.getdbt.com/api/v2/accounts/1713/jobs/121379/run/)\nSTATUSCODE=$(echo "$RESPONSE" | jq \'.status.code\')\necho "$RESPONSE" | jq \'.status.user_message\'\nif [[ $STATUSCODE != 200 ]]; then echo "$RESPONSE" && bash -c "exit 1"; fi\n'